In [1]:
# !pip install tensorflow-gpu==2.3
# !pip install sklearn
# !pip install matplotlib

In [2]:
import numpy as np
np_load_old = np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)
import os
import matplotlib.pyplot as plt
# os.environ['CUDA_VISIBLE_DEVICES'] = '1'
# os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [3]:
import tensorflow
import tensorflow as tf
from tensorflow import keras

In [4]:
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [6]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 2605532109450018820,
 name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 11519315969332317648
 physical_device_desc: "device: XLA_CPU device",
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 3050779444
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 11433440636289079272
 physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5",
 name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 2241923138843759958
 physical_device_desc: "device: XLA_GPU device"]

In [7]:
file_dir1 = 'C:/Users/BVer/are_you_ok/preprocessing'

In [8]:
raw_data = np.load(file_dir1 + '/p_video/' + 'raw_kicking_k1.mp4_.npy')

In [9]:
raw_data.shape

(47,)

In [10]:
raw_data[0][0][8]

0.0

In [11]:
from tensorflow.keras.models import load_model

right_arm_model = load_model('./models/right_arm_model.h5')
left_arm_model = load_model('./models/left_arm_model.h5')
right_leg_model = load_model('./models/right_leg_model.h5')
left_leg_model = load_model('./models/left_leg_model.h5')

In [57]:
def pred_multiple(raw_data):
    detected_humans = [] #[human][index, part_seq, part_nan_count, part_action]
    for object_at_frame in range(len(raw_data)): #프레임 단위 반복
        for object_at in range(len(raw_data[object_at_frame])): #사람 단위 반복
            detect_state = 0
            for detect_seq in detected_humans:
                if raw_data[object_at_frame][object_at][8] == detect_seq[0]: #해당 인덱스의 사람이 존재하는지 검사
                    pred_raw_part(raw_data[object_at_frame][object_at], detect_seq)
                    detect_state = 1
                    break
            if detect_state == 0:
                print('new')
                detected_humans.append([raw_data[object_at_frame][object_at][8], [], 0, 0, [], 0, 0, [], 0, 0, [], 0, 0])
                pred_raw_part(raw_data[object_at_frame][object_at], detected_humans[-1])
               

In [19]:
raw_data[0][0]

array([78.95265198, 11.04735088, 78.63735962,  0.21171729,  5.96600342,
        8.67344761,  9.69378281, 14.8219471 ,  0.        ])

In [54]:
def pred_raw_part(object_n, human_seq):
    object_n = object_n[:-1]
    seq_length = 30

    actions = ['assult', 'normal']
    
#     right_arm_seq = [] #2,3
#     left_arm_seq = [] #4,5
#     right_leg_seq = [] #7,8
#     left_leg_seq = [] #10,11

#     right_arm_action = 0
#     left_arm_action = 0
#     right_leg_action = 0      
#     left_leg_action = 0

#     right_arm_nan = 0
#     left_arm_nan = 0
#     right_leg_nan = 0      
#     left_leg_nan = 0
    
    nan_skip_count = 10 #오차 허용 프레임
        
    human_seq[2] = take_not_nan(object_n[0], object_n[1], human_seq[1], human_seq[2])
    human_seq[5] = take_not_nan(object_n[2], object_n[3], human_seq[4], human_seq[5])
    human_seq[8] = take_not_nan(object_n[4], object_n[5], human_seq[7], human_seq[8])
    human_seq[11] = take_not_nan(object_n[6], object_n[7], human_seq[10], human_seq[11])
        
    for process_count in range(4):
        if human_seq[process_count * 3 + 2] == 15:
            print("cleared seq")
            human_seq[process_count * 3 + 1] = []
            human_seq[process_count * 3 + 3] = 0
            
        
#         right_arm_action = pred_part(right_arm_seq, right_arm_nan, right_arm_model, right_arm_action, 'right_arm')
#         left_arm_action = pred_part(left_arm_seq, left_arm_nan, left_arm_model, left_arm_action, 'left_arm')
#         right_leg_action = pred_part(right_leg_seq, right_leg_nan, right_leg_model, right_leg_action, 'right_leg')
#         left_leg_action = pred_part(left_leg_seq, left_leg_nan, left_leg_model, left_leg_action, 'left_leg')

In [55]:
def take_not_nan(key1, key2, part_seq, part_nan):
    if not np.isnan(key1) and not np.isnan(key2):
        part_seq.append([key1, key2])
        return 0
    else:
        part_nan += 1
        if part_nan > 10: #오차 허용 프레임
            return 15
        else:
            return part_nan
        
def pred_part(part_seq, part_nan, part_model, part_action, part_name):
    seq_length = 30
    if len(part_seq) > seq_length and part_nan == 0:
        input_data = np.expand_dims(np.array(part_seq[-seq_length:], dtype=np.float32), axis=0)
        y_pred = part_model.predict(input_data).squeeze()

        i_pred = int(np.argmax(y_pred))
        conf = y_pred[i_pred]
        
#         print(conf, i_pred, part_name)

        if conf > 0.5 and i_pred == 0: #신뢰도
            part_action += 1
            print(part_action)
            if part_action > 2:
                print(part_name, '  detect  assult ', conf)
            return part_action
        else:
            return 0
    return part_action

In [58]:
pred_multiple(raw_data)

new
new


In [53]:
pred_raw_part(raw_data)

1
2
3
right_arm   detect  assult  0.7269128
4
right_arm   detect  assult  0.7384429
1
2
3
right_arm   detect  assult  0.6863602
4
right_arm   detect  assult  0.7667644
1
2
3
right_arm   detect  assult  0.6827853
4
right_arm   detect  assult  0.65501714
5
right_arm   detect  assult  0.60551846
1
1
2
1
1
1
1
1
2
1
1
2
3
right_leg   detect  assult  0.7919584
4
right_leg   detect  assult  0.7234119
5
right_leg   detect  assult  0.70603997
6
right_leg   detect  assult  0.768806
7
right_leg   detect  assult  0.66453093
1
2
1
2
3
left_leg   detect  assult  0.56204003
4
left_leg   detect  assult  0.5473652
5
left_leg   detect  assult  0.8874635
6
left_leg   detect  assult  0.80719084
7
left_leg   detect  assult  0.74128383
8
left_leg   detect  assult  0.68282866
9
left_leg   detect  assult  0.5729641
1
1
1
2
3
right_leg   detect  assult  0.517606
1
2
3
right_arm   detect  assult  0.56081927
1
2
3
right_arm   detect  assult  0.6308519
4
right_arm   detect  assult  0.84621716
5
right_arm   detec